## Process submitted sample audio clips ("fucks")

### Fusing sample audio files together

In [68]:
import os
from os import listdir

sample_list = os.listdir('samples/')

comb = silence_chunk
for i, sample in enumerate(sample_list):
    one = AudioSegment.from_file("samples/" + sample)
    comb = comb + silence_chunk + one
    
comb.export('fucks_all.wav', format = 'wav')

### Splitting audio files on silence

In [ ]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)

song = AudioSegment.from_file("misc_audio/fucks_all.wav")

#split track where silence is 200ms+ and get chunks

chunks = split_on_silence(song, 
    # must be silent for at least x ms
    min_silence_len=300,

    # consider it silent if quieter than threshold dBFS
    silence_thresh=-36 
)

#Process each chunk per requirements
for i, chunk in enumerate(chunks):
    #Create 0.5 seconds silence chunk
    silence_chunk = AudioSegment.silent(duration=100)

    #Add  0.5 sec silence to beginning and end of audio chunk
    audio_chunk = silence_chunk + chunk + silence_chunk

    #Normalize each audio chunk
    normalized_chunk = match_target_amplitude(audio_chunk, -20.0)

    #Export audio chunk with new bitrate
    print("exporting fuck{0}.mp3".format(i) )
    normalized_chunk.export(".//train/fucks_wav/fuck{0}.wav".format(i), bitrate='192k', format="wav")

## Make non-fuck clips of comparable length

### Select N random clips

In [1]:
import random
import pandas as pd

# import transcripts to a dataframe
meta_train = pd.read_csv('cv_corpus_v1/cv-valid-train.csv')

n = 1000
randoms = random.sample(range(0, len(meta_train)), n)

selection = meta_train.iloc[randoms, [0,1]]
selection.sort_index(inplace=True)
selection = selection.reset_index()

### Crop 1s from 1000 mozilla mp3 files & copy over as wav

In [2]:
from pydub import AudioSegment

for x in selection.filename:
    file_path = str('cv_corpus_v1/' + x)
    file_name = x[-17:-4]
    clip = AudioSegment.from_file(file_path)
    
    duration = len(clip)
    crop_start = random.sample(range(0, duration - 1000), 1)[0]
    crop_end = crop_start + random.sample(range(300, 1000), 1)[0]
    
    silence_chunk = AudioSegment.silent(duration=100)
    cropped_clip = silence_chunk + clip[crop_start : crop_end] + silence_chunk
    
    cropped_clip.export('train/other_wav/' + str(file_name) + '-cropped.wav', format = 'wav')

## Split my test read into 2s clips

In [1]:
from pydub import AudioSegment

test_all = AudioSegment.from_file('misc_audio/fuck_history.wav')
len(test_all)

start_times = []
for i in range(0, 346000, 500):
    start_times.append(i)
    
for i in start_times:
    clip = test_all[i:(i+500)]
    silence_chunk = AudioSegment.silent(duration=100)
    cropped_clip = silence_chunk + clip + silence_chunk
    cropped_clip.export('test/test_wav/test_' + str(i) + '.wav', format = 'wav')

In [2]:
62000/4

15500.0